In [ ]:
import sys

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import time
from sklearn.decomposition import PCA
import pandas as pd

### load data

In [ ]:
holdOutSamples=['HV1','P22','P14','P27']

sourceDir='/data/xinyi/c2p/data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']

outSize=128
savename='pathCentered_'+str(outSize)

imgsC_all=None
imgsP_all=None
imgNames_all=None
proteinNames=None
pID_all=None
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID in holdOutSamples:
                print('hold out: '+pID)
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)

            imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
            proteinNames_curr=np.array([])
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                imgP[s_start:s_end]=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])
                proteinNames_curr=np.concatenate((proteinNames_curr,np.repeat(stain_list[s],s_end-s_start)))
            
            if pID_all is None:
                pID_all=np.repeat(pID,img.shape[0])
                imgsC_all=img[allIdx_all,[0]]
                imgNames_all=imgNames[allIdx_all]
                proteinNames=proteinNames_curr
                imgsP_all=imgP
            else:
                pID_all=np.concatenate((pID_all,np.repeat(pID,img.shape[0])))
                imgsC_all=np.concatenate((imgsC_all,img[allIdx_all,[0]]),axis=0)
                imgNames_all=np.concatenate((imgNames_all,imgNames[allIdx_all]))
                proteinNames=np.concatenate((proteinNames,proteinNames_curr))
                imgsP_all=np.concatenate((imgsP_all,imgP),axis=0)
imgsC_all=imgsC_all.reshape(imgsC_all.shape[0],1,imgsC_all.shape[1],imgsC_all.shape[2])

In [ ]:
imgsC_val=None
imgsP_val=None
imgNames_val=None
proteinNames_val=None
pID_val=None
imgsP_val_all=None
imgsP_val_all_names=None
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID not in holdOutSamples:
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)

            imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
            imgP_all=np.zeros((img.shape[0],3,img.shape[2],img.shape[3]))
            proteinNames_val_curr=np.array([])
            imgsP_val_all_names_curr=None
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                imgP[s_start:s_end]=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])
                proteinNames_val_curr=np.concatenate((proteinNames_val_curr,np.repeat(stain_list[s],s_end-s_start)))
                imgP_all[s_start:s_end,:img.shape[1]-1]=img[allIdx_all[s_start:s_end],1:].reshape(s_end-s_start,img.shape[1]-1,img.shape[2],img.shape[3])
                if imgsP_val_all_names_curr is None:
                    imgsP_val_all_names_curr=np.tile(stain_list[1:],(s_end-s_start,1))
                else:
                    imgsP_val_all_names_curr=np.concatenate((imgsP_val_all_names_curr,np.tile(stain_list[1:],(s_end-s_start,1))),axis=0)
            if imgsP_val_all_names_curr.shape[1]==2:
                imgsP_val_all_names_curr=np.hstack((imgsP_val_all_names_curr,np.repeat('None',imgsP_val_all_names_curr.shape[0]).reshape(-1,1)))
            if pID_val is None:
                pID_val=np.repeat(pID,img.shape[0])
                imgsC_val=img[allIdx_all,[0]]
                imgNames_val=imgNames[allIdx_all]
                proteinNames_val=proteinNames_val_curr
                imgsP_val=imgP
                imgsP_val_all=imgP_all
                imgsP_val_all_names=imgsP_val_all_names_curr
            else:
                pID_val=np.concatenate((pID_val,np.repeat(pID,img.shape[0])))
                imgsC_val=np.concatenate((imgsC_val,img[allIdx_all,[0]]),axis=0)
                imgNames_val=np.concatenate((imgNames_val,imgNames[allIdx_all]))
                proteinNames_val=np.concatenate((proteinNames_val,proteinNames_val_curr))
                imgsP_val=np.concatenate((imgsP_val,imgP),axis=0)
                imgsP_val_all=np.concatenate((imgsP_val_all,imgP_all),axis=0)
                imgsP_val_all_names=np.concatenate((imgsP_val_all_names,imgsP_val_all_names_curr),axis=0)
imgsC_val=imgsC_val.reshape(imgsC_val.shape[0],1,imgsC_val.shape[1],imgsC_val.shape[2])

In [ ]:
nProt=np.unique(proteinNames).size
pnames,revIdx,pCounts=np.unique(proteinNames,return_inverse=True,return_counts=True)
plabels=torch.tensor(np.arange(pnames.size)[revIdx]).long()

In [ ]:
nProt_val=np.unique(proteinNames_val).size
pnames_val,revIdx_val,pCounts_val=np.unique(proteinNames_val,return_inverse=True,return_counts=True)
plabels_val=torch.tensor(np.arange(pnames_val.size)[revIdx_val]).long()

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

### cell inpainting model converted from tensorflow/keras

In [ ]:
class Pair_Model(nn.Module):
    def __init__(self):
        super(Pair_Model, self).__init__()
        # First two conv layers of source cell encoder
        self.encode_x=nn.Sequential(
            nn.Conv2d(2, 96, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size=(2, 2)),

            nn.Conv2d(96, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(2, 2)),

            # Last three conv layers of source cell encoder
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(384),
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(384),
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256),
        )
        
        self.encode_y=nn.Sequential(
            # First two conv layers of target marker encoder
            nn.Conv2d(1, 16, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=(2, 2)),

            nn.Conv2d(16, 32, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(2, 2)),
            
            # Last conv layer of target marker encoder
            nn.Conv2d(32, 32, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )
        
        self.decoder=nn.Sequential(
            nn.Conv2d(256+32, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(256, 96, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(96, 1, kernel_size=(1, 1))
        )

    def forward(self, x, y_rfp):
        x=self.encode_x(x)
        y_rfp=self.encode_y(y_rfp)
        
        return self.decoder(torch.cat((x, y_rfp), dim=1))



In [ ]:
seed=3
epochs=5001
saveFreq=100
lr=1e-4
batchsize=128

loss_x=torch.nn.MSELoss()

In [ ]:
printFreq=1
valIdx=np.arange(imgsC_val.shape[0])
def train(epoch):
    t=time.time()
    model.train()
    loss_all=0
    ntrainBatches_all=0
    for pi in range(nProt):
        print(pnames[pi])
        trainIdx=np.arange(np.sum(proteinNames==pnames[pi]))
        sourceIdx=np.arange(proteinNames.size)[proteinNames==pnames[pi]]
        ntrainBatches=int(np.ceil(trainIdx.shape[0]/batchsize))
        for i in range(ntrainBatches):
            trainIdx_i=trainIdx[i*batchsize:min((i+1)*batchsize,trainIdx.shape[0])]
            trainSource_dna=imgsC_all[sourceIdx[trainIdx_i]]
            trainSource_protein=imgsP_all[sourceIdx[trainIdx_i]]
            trainSource=torch.tensor(np.concatenate((trainSource_dna,trainSource_protein),axis=1)).cuda().float()
            targetIdx=np.concatenate((trainIdx[:i*batchsize],trainIdx[min((i+1)*batchsize,trainIdx.shape[0]):]))
            np.random.seed(epoch)
            targetIdx=np.random.choice(targetIdx,trainIdx_i.size,replace=False)
            trainTarget_dna=torch.tensor(imgsC_all[sourceIdx[targetIdx]]).cuda().float()
            trainTarget_protein=torch.tensor(imgsP_all[sourceIdx[targetIdx]]).cuda().float()
            
            optimizer.zero_grad()
            pred=model(trainSource,trainTarget_dna)
            
            loss=loss_x(pred,trainTarget_protein)
            loss_all+=loss.item()
            loss.backward()
            optimizer.step()
        ntrainBatches_all+=ntrainBatches
    loss_all=loss_all/ntrainBatches_all
    
    with torch.no_grad():
        model.eval()
        
        loss_val_all=0
        nvalBatches_all=0
        for pi in range(nProt):
            print(pnames[pi])
            valIdx=np.arange(np.sum(proteinNames_val==pnames[pi]))
            sourceIdx=np.arange(proteinNames_val.size)[proteinNames_val==pnames[pi]]
            nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
            for i in range(nvalBatches):
                valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
                valSource_dna=imgsC_val[sourceIdx[valIdx_i]]
                valSource_protein=imgsP_val[sourceIdx[valIdx_i]]
                valSource=torch.tensor(np.concatenate((valSource_dna,valSource_protein),axis=1)).cuda().float()
                targetIdx=np.concatenate((valIdx[:i*batchsize],valIdx[min((i+1)*batchsize,valIdx.shape[0]):]))
                np.random.seed(epoch)
                targetIdx=np.random.choice(targetIdx,valIdx_i.size,replace=False)
                valTarget_dna=torch.tensor(imgsC_val[sourceIdx[targetIdx]]).cuda().float()
                valTarget_protein=torch.tensor(imgsP_val[sourceIdx[targetIdx]]).cuda().float()

                pred=model(valSource,valTarget_dna)

                loss=loss_x(pred,valTarget_protein)
                loss_val_all+=loss.item()
            nvalBatches_all+=nvalBatches
        loss_val_all=loss_val_all/nvalBatches_all
    print('Epoch: {:04d}'.format(epoch),
          'loss_train: {:.4f}'.format(loss_all),
          'loss_val: {:.4f}'.format(loss_val_all),
          'time: {:.4f}s'.format(time.time() - t))
    return loss_all,loss_val_all

In [ ]:
name_train='inpainting'
modelname_train='benchmark'
logsavepath_train=os.path.join('/data/xinyi/c2p/log/',modelname_train,name_train)
modelsavepath_train=os.path.join('/data/xinyi/c2p/models/',modelname_train,name_train)
plotsavepath_train=os.path.join('/data/xinyi/c2p/plots/',modelname_train,name_train)

if not os.path.exists(os.path.join('/data/xinyi/c2p/log/',modelname_train)):
    os.mkdir(os.path.join('/data/xinyi/c2p/log/',modelname_train))
    os.mkdir(os.path.join('/data/xinyi/c2p/models/',modelname_train))
    os.mkdir(os.path.join('/data/xinyi/c2p/plots/',modelname_train))
if not os.path.exists(logsavepath_train):
    os.mkdir(logsavepath_train)
if not os.path.exists(modelsavepath_train):
    os.mkdir(modelsavepath_train)
if not os.path.exists(plotsavepath_train):
    os.mkdir(plotsavepath_train)


In [ ]:
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.enabled = True

model=Pair_Model()
model.cuda()
optimizer=torch.optim.Adam(model.parameters(),lr=lr,betas=(0.5, 0.999)) #setting betas1 to 0.5 to be consistent with https://github.com/alexxijielu/paired_cell_inpainting/

train_loss=[np.inf]*epochs
val_loss=[np.inf]*epochs
t_ep=time.time()

epCounts=0
for ep in range(epochs):
    train_loss[ep],val_loss[ep]=train(ep)

    if ep>100 and val_loss[ep]>=val_loss[ep-100]:
        epCounts+=1
    else:
        epCounts=0

    if epCounts>100:
        break


    if ep%saveFreq == (saveFreq-1):
        torch.save(model.cpu().state_dict(),os.path.join(modelsavepath_train,str(ep)+'.pt'))

    model.cuda()
    torch.cuda.empty_cache()
print(' total time: {:.4f}s'.format(time.time() - t_ep))


with open(os.path.join(logsavepath_train,'train_loss'), 'wb') as output:
    pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath_train,'val_loss'), 'wb') as output:
    pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
                   

plt.plot(np.arange(epochs),train_loss)
plt.plot(np.arange(epochs),val_loss)
plt.legend(['train','val'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_train,'loss_seed3.jpg'))
plt.show()




In [ ]:
with open(os.path.join(logsavepath_train,'train_loss'), 'wb') as output:
    pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath_train,'val_loss'), 'wb') as output:
    pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
                   

plt.plot(np.arange(epochs),train_loss)
plt.plot(np.arange(epochs),val_loss)
plt.ylim(0,0.02)
plt.legend(['train','val'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_train,'loss_seed3.jpg'))
plt.show()

In [ ]:
torch.save(model.cpu().state_dict(),os.path.join(modelsavepath_train,str(ep)+'.pt'))
model.cuda()

In [ ]:
model=Pair_Model()
model.cuda()
ep=1063
model.load_state_dict(torch.load(os.path.join(modelsavepath_train,str(ep)+'.pt')))


In [ ]:
ep=1063
model.load_state_dict(torch.load(os.path.join(modelsavepath_train,str(ep)+'.pt')))
with torch.no_grad():
    model.eval()

    loss_val_all=0
    nvalBatches_all=0
    for pi in range(nProt):
        print(pnames[pi])
        valIdx=np.arange(np.sum(proteinNames_val==pnames[pi]))
        sourceIdx=np.arange(proteinNames_val.size)[proteinNames_val==pnames[pi]]
        nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
            valSource_dna=imgsC_val[sourceIdx[valIdx_i]]
            valSource_protein=imgsP_val[sourceIdx[valIdx_i]]
            valSource=torch.tensor(np.concatenate((valSource_dna,valSource_protein),axis=1)).cuda().float()
            targetIdx=np.concatenate((valIdx[:i*batchsize],valIdx[min((i+1)*batchsize,valIdx.shape[0]):]))
            np.random.seed(ep)
            targetIdx=np.random.choice(targetIdx,valIdx_i.size,replace=False)
            valTarget_dna=torch.tensor(imgsC_val[sourceIdx[targetIdx]]).cuda().float()
            valTarget_protein=torch.tensor(imgsP_val[sourceIdx[targetIdx]]).cuda().float()

            pred=model(valSource,valTarget_dna)

            loss=loss_x(pred,valTarget_protein)
            loss_val_all+=loss.item()
            
            for j in range(3):
                print(j)
                print(proteinNames_val[sourceIdx[valIdx_i]][j])
                print(proteinNames_val[sourceIdx[targetIdx]][j])
                plt.imshow(valTarget_protein[j][0].cpu().detach().numpy())
                plt.show()
                plt.imshow(pred[j][0].cpu().detach().numpy())
                plt.show()
                plt.imshow(valTarget_dna[j][0].cpu().detach().numpy())
                plt.show()
                print(loss_x(valTarget_protein[j][0],pred[j][0]).item())

                plt.imshow(valSource_dna[j][0])
                plt.show()
                plt.imshow(valSource_protein[j][0])
                plt.show()
        nvalBatches_all+=nvalBatches
    loss_val_all=loss_val_all/nvalBatches_all

In [ ]:
loss_val_all

In [ ]:
def percentileNorm(img_c):
#     intensity,intCounts=np.unique(img_c,return_counts=True)
#     modeint=intensity[np.argmax(intCounts)]
    modeint=np.percentile(img_c,25)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/np.max(img_c)
    print(modeint)
    return img_c

def modeSub(img_c):
    intensity,intCounts=np.unique(img_c,return_counts=True)
    modeint=intensity[np.argmax(intCounts)]
#     modeint=np.percentile(img_c,75)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/np.max(img_c)
    print(modeint)
    return img_c

def modeSub_torch(img_c):
    intensity,intCounts=torch.unique(img_c,return_counts=True)
    modeint=intensity[torch.argmax(intCounts)]
#     modeint=np.percentile(img_c,75)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/torch.max(img_c)
#     print(modeint)
    return img_c

In [ ]:
#plotting prediction of  all proteins
plottingIdx=np.array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,  328,
                      329,  330,  331,  332,  333,  334,  335,  336,  337,  656,  657,
                      658,  659,  660,  661,  662,  663,  664,  665,  984,  985,  986,
                      987,  988,  989,  990,  991,  992,  993, 1312, 1313, 1314, 1315,
                      1316, 1317, 1318, 1319, 1320, 1321, 1640, 1641, 1642, 1643, 1644,
                      1645, 1646, 1647, 1648, 1649, 1968, 1969, 1970, 1971, 1972, 1973,
                      1974, 1975, 1976, 1977, 2296, 2297, 2298, 2299, 2300, 2301, 2302,
                      2303, 2304, 2305, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 2631,
                      2632, 2633, 2952, 2953, 2954, 2955, 2956, 2957, 2958, 2959, 2960,
                     2961, 3280, 3281, 3282, 3283, 3284, 3285, 3286, 3287, 3288, 3289])
with torch.no_grad():
    model.eval()


    for i in range(plottingIdx.size):
        print(i)
        print('input img',proteinNames_val[plottingIdx][i])
        
        
        valtarget_protein=torch.tensor(imgsP_val[[plottingIdx[i]]]).cuda().float()
        valtarget_dna=torch.tensor(imgsC_val[[plottingIdx[i]]]).cuda().float()
        valInput_ID_orig=plabels_val[[plottingIdx[i]]].cuda()
        valIdx_i=torch.tensor([plottingIdx[i]])
        valAllProteins=imgsP_val_all[plottingIdx[i]]
        valAllProteins_names=imgsP_val_all_names[plottingIdx[i]]
        
        plt.imshow(modeSub(valtarget_dna[0][0].cpu().detach().numpy()))
        plt.show()
        for pidx in range(3):
            if valAllProteins_names[pidx]=='None':
                continue
            print('True ',valAllProteins_names[pidx])
            plt.imshow(percentileNorm(valAllProteins[pidx]))
            plt.show()
        
        for pidx in range(pnames.size):
            print(pnames[pidx])
            
            sourceIdx=np.concatenate((np.arange(proteinNames_val.size)[:plottingIdx[i]],np.arange(proteinNames_val.size)[plottingIdx[i]+1:]))
            np.random.seed(i)
            sourceIdx=np.random.choice(sourceIdx[np.concatenate((proteinNames_val[:plottingIdx[i]]==pnames[pidx],proteinNames_val[plottingIdx[i]+1:]==pnames[pidx]))],1)
            valSource_dna=imgsC_val[sourceIdx]
            valSource_protein=imgsP_val[sourceIdx]
            
            
            valSource=torch.tensor(np.concatenate((valSource_dna,valSource_protein),axis=1)).cuda().float()
            

            pred=model(valSource,valtarget_dna)
            


            plt.imshow(percentileNorm(pred[0][0].cpu().detach().numpy()))
            plt.show()
            plt.imshow(modeSub(pred[0][0].cpu().detach().numpy()))
            plt.show()

            



In [ ]:
imgsC_val_allProt={}
imgsP_val_allProt={}
imgsP_val_allProt_input={}
imgNames_val_allProt={}
pID_val_allProt={}
conditions_val_allProt={}
proteinNames_val_allProt={}
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID not in holdOutSamples:
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)
                
#             imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
#             proteinNames_val_curr=np.array([])
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                proteinNames_val_curr=np.repeat(stain_list[s],s_end-s_start)
                imgP=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])

                for sother in range(1,len(stain_list)):
                    if sother==s:
                        continue
                    if stain_list[sother] not in imgsP_val_allProt.keys():
                        pID_val_allProt[stain_list[sother]]=np.repeat(pID,s_end-s_start)
                        imgsC_val_allProt[stain_list[sother]]=img[allIdx_all[s_start:s_end],[0]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])
                        imgNames_val_allProt[stain_list[sother]]=imgNames[allIdx_all[s_start:s_end]]
                        imgsP_val_allProt[stain_list[sother]]=img[allIdx_all[s_start:s_end],[sother]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])
                        conditions_val_allProt[stain_list[sother]]=np.repeat(condition_i,s_end-s_start)
                        proteinNames_val_allProt[stain_list[sother]]=proteinNames_val_curr
                        imgsP_val_allProt_input[stain_list[sother]]=imgP
                    else:
                        pID_val_allProt[stain_list[sother]]=np.concatenate((pID_val_allProt[stain_list[sother]],np.repeat(pID,s_end-s_start)))
                        imgsC_val_allProt[stain_list[sother]]=np.concatenate((imgsC_val_allProt[stain_list[sother]],img[allIdx_all[s_start:s_end],[0]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])),axis=0)
                        imgNames_val_allProt[stain_list[sother]]=np.concatenate((imgNames_val_allProt[stain_list[sother]],imgNames[allIdx_all[s_start:s_end]]))
                        imgsP_val_allProt[stain_list[sother]]=np.concatenate((imgsP_val_allProt[stain_list[sother]],img[allIdx_all[s_start:s_end],[sother]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])),axis=0)
                        conditions_val_allProt[stain_list[sother]]=np.concatenate((conditions_val_allProt[stain_list[sother]],np.repeat(condition_i,s_end-s_start)))
                        proteinNames_val_allProt[stain_list[sother]]=np.concatenate((proteinNames_val_allProt[stain_list[sother]],proteinNames_val_curr))
                        imgsP_val_allProt_input[stain_list[sother]]=np.concatenate((imgsP_val_allProt_input[stain_list[sother]],imgP),axis=0)


In [ ]:
#prediction loss of all proteins
loss_l1=torch.nn.L1Loss()
with torch.no_grad():
    model.eval()

    

    for pidx in range(pnames.size):
        print(pnames[pidx])
        
        plabels_orig=torch.zeros(proteinNames_val_allProt[pnames[pidx]].size,dtype=int)
        for pidx_label in range(pnames.size):
            plabels_orig[proteinNames_val_allProt[pnames[pidx]]==pnames[pidx_label]]=pidx_label
        
        valInput_ID_single=torch.tensor([pidx]).cuda()
        
        valIdx_p=np.arange(imgsP_val_allProt[pnames[pidx]].shape[0])
        loss_x_valShared_all_protein=0
        loss_x_val_all_protein=0
        nvalBatches=int(np.ceil(valIdx_p.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx_i=valIdx_p[i*batchsize:min((i+1)*batchsize,valIdx_p.shape[0])]
            valtarget_protein=torch.tensor(imgsP_val_allProt_input[pnames[pidx]][valIdx_i]).cuda().float()
            valtarget_protein_pred=torch.tensor(imgsP_val_allProt[pnames[pidx]][valIdx_i]).cuda().float()
            valtarget_dna=torch.tensor(imgsC_val_allProt[pnames[pidx]][valIdx_i]).cuda().float()
            valInput_ID=torch.repeat_interleave(valInput_ID_single,valIdx_i.shape[0]).cuda()
            valInput_ID_orig=plabels_orig[valIdx_i].cuda()

            
            
            sourceIdx=np.concatenate((np.arange(proteinNames_val_allProt[pnames[pidx]].size)[:i*batchsize],np.arange(proteinNames_val_allProt[pnames[pidx]].size)[min((i+1)*batchsize,valIdx_p.shape[0]):]))
            np.random.seed(i)
            sourceIdx=np.random.choice(sourceIdx,valIdx_i.size,replace=False)
            valSource_dna=imgsC_val_allProt[pnames[pidx]][sourceIdx]
            valSource_protein=imgsP_val_allProt_input[pnames[pidx]][sourceIdx]
            
            
            valSource=torch.tensor(np.concatenate((valSource_dna,valSource_protein),axis=1)).cuda().float()
            

            pred=model(valSource,valtarget_dna)
            
            predmin=torch.min(pred,dim=1,keepdim=True)[0]
            predmin=torch.min(pred,dim=2,keepdim=True)[0]
            predmin=torch.min(pred,dim=3,keepdim=True)[0]
            predmax=torch.max(pred,dim=1,keepdim=True)[0]
            predmax=torch.max(pred,dim=2,keepdim=True)[0]
            predmax=torch.max(pred,dim=3,keepdim=True)[0]
            pred=(pred-predmin)/(predmax-predmin)
            
            loss_x_val_protein=loss_l1(modeSub_torch(pred),valtarget_protein_pred)
            if np.isfinite(loss_x_val_protein.item()):
                loss_x_val_all_protein+=loss_x_val_protein.item()


        loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
        print(loss_x_val_all_protein)